In [6]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
# from kaggle_datasets import KaggleDatasets


In [7]:
IMAGE_SIZE = 32
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 15

In [8]:
dataset = tf.keras.preprocessing.image_dataset_from_directory (
 "COVID-19_Lung_CT_Scans", shuffle = True, image_size = (IMAGE_SIZE, IMAGE_SIZE), batch_size = BATCH_SIZE)

Found 8439 files belonging to 2 classes.


In [9]:
class_names = dataset.class_names
class_names

['COVID-19', 'Non-COVID-19']

In [10]:
len(dataset)

264

In [11]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch.shape)
#     print(image_batch[0].numpy())
    print(label_batch.numpy())

(32, 32, 32, 3)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [12]:
train_size = 0.8
len(dataset)*train_size

211.20000000000002

In [13]:
train_ds = dataset.take(211)
len(train_ds)

211

In [14]:
temp_ds = dataset.skip(211)
len(temp_ds)


53

In [15]:
val_size = 0.1
len(dataset)*val_size
val_ds = temp_ds.take(25)
len(val_ds)


25

In [16]:
test_ds = temp_ds.skip(25)
len(test_ds)

28

In [17]:
def get_dataset_partitions_tf(ds, train_split = 0.8, val_split = 0.1 , test_split = 0.1, shuffle = True, shuffle_size = 10000):
    
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds 

In [18]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [19]:
len(train_ds)

211

In [20]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [21]:
# rollnumbers_based_CNN_model = 361174 --> 471163 , 

input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

def createModel():
    model = Sequential()
    # The first two layers with 32 filters of window size 3x3 as 4-1 =3
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    
    model.add(Conv2D(64, (1, 1), padding='same', activation='relu'))
    model.add(Conv2D(64, (1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    return model

In [22]:
model1 = createModel()
batch_size = 32
epochs = 15
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        2112      
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        4160      
                                                        

In [23]:
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
history = model1.fit(train_ds, epochs = epochs, batch_size = batch_size, verbose = 1, validation_data = val_ds)

Epoch 1/15
211/211 [==============================] - 70s 221ms/step - loss: 0.7747 - accuracy: 0.8627 - val_loss: 0.3028 - val_accuracy: 0.9038
Epoch 2/15
211/211 [==============================] - 27s 125ms/step - loss: 0.3302 - accuracy: 0.8876 - val_loss: 0.2709 - val_accuracy: 0.9038
Epoch 3/15
211/211 [==============================] - 26s 125ms/step - loss: 0.2668 - accuracy: 0.9036 - val_loss: 0.1475 - val_accuracy: 0.9471
Epoch 4/15
211/211 [==============================] - 25s 117ms/step - loss: 0.1936 - accuracy: 0.9352 - val_loss: 0.1797 - val_accuracy: 0.9459
Epoch 5/15
211/211 [==============================] - 27s 127ms/step - loss: 0.1667 - accuracy: 0.9447 - val_loss: 0.1032 - val_accuracy: 0.9639
Epoch 6/15
211/211 [==============================] - 27s 126ms/step - loss: 0.1396 - accuracy: 0.9565 - val_loss: 0.0820 - val_accuracy: 0.9748
Epoch 7/15
211/211 [==============================] - 26s 123ms/step - loss: 0.1270 - accuracy: 0.9580 - val_loss: 0.1120 - val_ac

In [25]:
model1.evaluate(train_ds)

211/211 [==============================] - 9s 43ms/step - loss: 0.0327 - accuracy: 0.9893


[0.032701361924409866, 0.9893222451210022]

In [26]:
model1.evaluate(val_ds)

26/26 [==============================] - 1s 41ms/step - loss: 0.0404 - accuracy: 0.9892


[0.040383920073509216, 0.989182710647583]

In [27]:
model1.evaluate(test_ds)

27/27 [==============================] - 19s 42ms/step - loss: 0.0443 - accuracy: 0.9861


[0.04430871829390526, 0.9861111044883728]

In [29]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
print('training loss', train_loss)
print('val_loss', val_loss)
print('train_acc', train_acc)
print('val_acc', val_acc)
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

training loss [0.7746715545654297, 0.3302414119243622, 0.26680612564086914, 0.1935776025056839, 0.1667155623435974, 0.13955119252204895, 0.12697631120681763, 0.11499515920877457, 0.10742764919996262, 0.09144790470600128, 0.10484741628170013, 0.08494939655065536, 0.08026254922151566, 0.06952022761106491, 0.06650924682617188]
val_loss [0.30282333493232727, 0.2709035575389862, 0.1475432813167572, 0.17967239022254944, 0.10322503000497818, 0.08197784423828125, 0.11202148348093033, 0.06843411177396774, 0.06540719419717789, 0.16542468965053558, 0.07328346371650696, 0.03928814083337784, 0.04119427874684334, 0.04308674857020378, 0.04038392752408981]
train_acc [0.8626723885536194, 0.8875871300697327, 0.903603732585907, 0.9351920485496521, 0.9446833729743958, 0.9565475583076477, 0.9580305218696594, 0.9642592072486877, 0.9655939340591431, 0.9721192121505737, 0.9664837718009949, 0.9736022353172302, 0.9736022353172302, 0.9787928462028503, 0.9808690547943115]
val_acc [0.9038461446762085, 0.9038461446

In [30]:
prediction = model1.predict(test_ds)

27/27 [==============================] - 1s 35ms/step


In [31]:
prediction[0]

array([9.9999863e-01, 1.3294973e-06, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.0875560e-38,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [32]:
import numpy as np

In [ ]:
plt.figure(figsize=(20,8))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"\n Actual:{class_names[labels[i]]} \n Prediction:{class_names[np.argmax(prediction[i])]}")
        plt.axis("off")